# Set up

In [ ]:
# Matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
# Numpy
import numpy as np
# Pandas
import pandas as pd
# Torch
import torch
from torch.utils.data import DataLoader
# Others
import random

In [ ]:
# Use GPU if available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

seed = 42  # A popular choice, but the specific number doesn't matter much

torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# If you are using CUDA, you might want to add:
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # For multi-GPU.
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Define dataset

In [ ]:
# I doubt we'll use this tbh

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, path):
        self.dataframe = pd.read_excel(path)
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        """
        Change inputs and outputs accordingly
        """

        # Select columns corresponding to the different inputs and outputs from the dataframe we just created.
        # And convert to PyTorch tensors with the right dtype
        x1 = torch.tensor(self.dataframe.iloc[idx, 0], dtype=torch.float64)
        x2 = torch.tensor(self.dataframe.iloc[idx, 1], dtype=torch.float64)
        y = torch.tensor(self.dataframe.iloc[idx, 2], dtype=torch.float64)
        
        # Assemble all input features in a single inputs tensor with 2 columns and rows for each sample in the dataset.
        inputs = torch.stack([x1, x2], dim=0)
        return inputs, y

In [ ]:
import torch
from torch.utils.data import random_split

# Dataset is gotten from customdataset above
dataset = CustomDataset("")

# Assuming `dataset` is your Dataset object
total_size = len(dataset)
train_size = int(total_size * 0.7)  # 70% for training
valid_size = int(total_size * 0.15)  # 15% for validation
test_size = total_size - train_size - valid_size  # Remaining 15% for testing

train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_size, valid_size, test_size])

from torch.utils.data import DataLoader

batch_size = 64  # Adjust as per your requirement

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Defining Blocks of layers

In [ ]:
class DenseReLU(torch.nn.Module):
    def __init__(self, n_x, n_y):
        super().__init__()
        # Define Linear layer using the nn.Linear()
        self.fc = torch.nn.Linear(n_x, n_y)
    
    def forward(self, x):
        # Wx + b operation
        # Using ReLU operation as activation after
        return torch.relu(self.fc(x))

# Defining Neural Network

In [ ]:
class DeepNeuralNet(torch.nn.Module):
    def __init__(self, n_x, n_h, n_y):
        super().__init__()
        # Define the correct number of Dense + ReLU layers based on n_h,
        # followed by one final Dense + Softmax layer
        values = [n_x] + n_h
        self.processing_layers = [DenseReLU(values[i], values[i + 1]) for i in range(len(values) - 1)]
        self.processing_layers += [DenseNoReLU(n_h[-1], n_y)]
        
        # Combine all layers
        # Important: note the * symbol before the list of layers in self.processing_layers
        # Not sure what it does? Check the *args and **kwargs concepts in Python.
        self.combined_layers = torch.nn.Sequential(*self.processing_layers)

    
    def forward(self, x):
        # Flatten images (transform them from 28x28 2D matrices to 784 1D vectors)
        x = x.view(x.size(0), -1)
        # Pass through all four layers
        out = self.combined_layers(x)
        return out

# Trainer function

Note: Test is the VALIDATION set here

Essentially, we want to pick the best validation score

In [ ]:
def trainer(model, train_loader, test_loader):
    
    # History for train acc, test acc
    train_accs = []
    test_accs = []
    
    # Define optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr = 1e-2)

    # Training model
    num_epochs = 10
    for epoch in range(num_epochs):

        # Go trough all samples in train dataset
        model.train()
        for i, (images, labels) in enumerate(train_loader):
            # Get from dataloader and send to device
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)

            # Compute loss
            loss = torch.nn.functional.cross_entropy(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Display
            if (i+1) % 25 == 0:
                print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Train Loss: {loss.item():.4f}')

        # Compute model train accuracy on test after all samples have been seen using test samples
        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for images, labels in train_loader:
                # Get images and labels from test loader
                images = images.to(device)
                labels = labels.to(device)

                # Forward pass and predict class using max
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)

                # Check if predicted class matches label and count numbler of correct predictions
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        # Compute final accuracy and display
        train_accuracy = correct/total
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Accuracy: {train_accuracy:.4f}')
        train_accs.append(train_accuracy)
        
        # Compute model test accuracy on test after all samples have been seen using test samples
        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for images, labels in test_loader:
                # Get images and labels from test loader
                images = images.to(device)
                labels = labels.to(device)

                # Forward pass and predict class using max
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)

                # Check if predicted class matches label and count numbler of correct predictions
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        # Compute final accuracy and display
        test_accuracy = correct/total
        print(f'Epoch [{epoch+1}/{num_epochs}], Test Accuracy: {test_accuracy:.4f}')
        test_accs.append(test_accuracy)
        
    # Return
    return train_accs, test_accs

# Hyperparameter tuning

We need to loop through and do some hyperparameter tuning here. Ideally with RandomSearch

Below is a placeholder display of data. We'll definitely have to change it

In [ ]:
plt.figure(figsize = (10, 7))
plt.plot(train_accs1, "r-", label = "Model 1 train acc.")
plt.plot(test_accs1, "b-", label = "Model 1 test acc.")
plt.plot(train_accs2, "r--", label = "Model 2 train acc.")
plt.plot(test_accs2, "b--", label = "Model 2 test acc.")
plt.plot(train_accs3, "r:", label = "Model 3 train acc.")
plt.plot(test_accs3, "b:", label = "Model 3 test acc.")
plt.legend(loc = "best")
plt.ylim([0.9, 1])
plt.show()

# Final Testing of dataset

In [ ]:
model.eval()  # Set the model to evaluation mode

with torch.no_grad():  # Temporarily set all the requires_grad flag to false
    correct = 0
    total = 0

    # Iterate over test data
    for data in test_loader:
        inputs, labels = data
        outputs = model(inputs)  # Get model predictions
        
        # For classification, the prediction is the index of the max log-probability
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Calculate the accuracy
    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the test dataset: {accuracy:.2f}%')